## Redis Guideline

In this notebook, we will explain what is stored and provided inside the redis database. What is the structure of each type of data including agent profile, relationship profile, environment profile, agentenv combo profile, and episodelog profile. Additionally, this notebook provides statistical information about how many datapoints are inside the provided database.

In [2]:
import os
import sys
sys.path.append("../")
os.environ["REDIS_OM_URL"] = "redis://:QzmCUD3C3RdsR@localhost:6379"
import rich
from collections import Counter
from sotopia.database.persistent_profile import (
    AgentProfile,
    EnvironmentProfile,
    RelationshipProfile,
)
from sotopia.database.logs import EpisodeLog
from sotopia.database.env_agent_combo_storage import EnvAgentComboStorage

## Relationship Profile

RelationshipProfile stores the information about different relationship between agents.

In [2]:
res_pks = RelationshipProfile.all_pks()
res_pks = list(res_pks)
print(len(res_pks))
res = []
for pk in res_pks:
    try:
        res.append(RelationshipProfile.get(pk=pk))
    except Exception:
        print("error")
        pass
res_relationships = [r.relationship for r in res]
Counter(res_relationships)

120


Counter({<RelationshipType.acquaintance: 2>: 31,
         <RelationshipType.friend: 3>: 24,
         <RelationshipType.know_by_name: 1>: 19,
         <RelationshipType.romantic_relationship: 4>: 16,
         <RelationshipType.family_member: 5>: 15,
         <RelationshipType.stranger: 0>: 15})

## Agents Profile

AgentProfile stores the information about each agent.

In [3]:
# obtain a specific agent
agents = AgentProfile.find(AgentProfile.first_name == "ss").all()
rich.print(agents)

[]

In [4]:
# find specific agnets
agents = AgentProfile.find(AgentProfile.gender == "Man", AgentProfile.age > 30)
agents = agents.all()
print(len(agents))
rich.print(agents[0])

15


AgentProfile(
    pk='01H5TNE5PFT9HH0WRT6W1NY5GZ',
    first_name='Leo',
    last_name='Williams',
    age=37,
    occupation='Dentist',
    gender='Man',
    gender_pronoun='He/him',
    public_info='Leo Williams often spends his time playing with his young daughter whom he cherishes dearly.',
    big_five='Openness to Experience - Low; Conscientiousness - High; Extraversion - High; Agreeableness - High; 
Neuroticism - Low',
    moral_values=['Care/Harm'],
    schwartz_personal_values=['Security'],
    personality_and_values="Leo Williams, despite being quiet, values security and care for others. He's a rational
decision-maker focussing on best outcomes.",
    decision_making_style='Rational',
    secret='He has a secret child from a previous relationship.',
    model_id='',
    mbti='ESFJ'
)

In [5]:
# obtain all agents' basic info
agent_pks = AgentProfile.all_pks()
agent_pks = list(agent_pks)
agents = []
for pk in agent_pks:
    try:
        agents.append(AgentProfile.get(pk=pk))
    except Exception:
        print("error")
        pass
print(len(agents))
rich.print(agents[0])

40


AgentProfile(
    pk='01H5TNE5PQ00AJVSSVB9V2VA9K',
    first_name='Eli',
    last_name='Dawson',
    age=52,
    occupation='Forensic psychiatrist',
    gender='Man',
    gender_pronoun='He/him',
    public_info='Eli Dawson, a forensic psychiatrist, is a history enthusiast and enjoys collecting vintage 
books.',
    big_five='Openness to Experience - High; Conscientiousness - High; Extraversion - Low; Agreeableness - Low; 
Neuroticism - Low',
    moral_values=['Liberty'],
    schwartz_personal_values=['Hedonism'],
    personality_and_values='Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and 
enjoyment. His decision-making is strategic, examining the wider context before making decisions.',
    decision_making_style='Strategic',
    secret='He secretly funds a college student',
    model_id='',
    mbti='INTJ'
)

In [6]:
agent_pks = AgentProfile.all_pks()
agent_pks = list(agent_pks)
print(len(agent_pks))

40


In [7]:
# Update agent's information
agents = AgentProfile.find(
    AgentProfile.first_name == "Ava", AgentProfile.last_name == "Martinez"
).all()[0]

In [8]:
agents.update(secret="Keeps their bisexuality a secret from her conservative family")

## Environment Profile

EnvironmentProfile stores the information about social scenario. 

In [9]:
# get all environments
all_envs = list(EnvironmentProfile.all_pks())
print(len(all_envs))
print(all_envs[:5])

90
['01H7VFHN6NYWSTWCZJE2DCQKTD', '01H7VFHND24JAWG23XMPYGG5HK', '01H7VFHNH8A88C4XJ7X4PVAHV4', '01H7VFHP43QEZA1WZB3B3J2D9X', '01H7VFHQ11NAMZS4A2RDGDB01V']


In [10]:
# get a specific environment profile
env_profile_id = all_envs[0]
env = EnvironmentProfile.get(env_profile_id)
rich.print(env)

EnvironmentProfile(
    pk='01H7VFHN6NYWSTWCZJE2DCQKTD',
    codename='facetime_etiquettes',
    source='social_chemistry',
    scenario='A couple having a conversation over FaceTime',
    agent_goals=[
        'Express feelings about the partner taking texts while on FaceTime (<extra_info>Extra information: You find
it disrespectful and it makes you feel unimportant.</extra_info>)',
        "Justify the act of texting while on FaceTime without hurting the partner's feelings (<extra_info>Extra 
information: You have some urgent and important messages to respond to.</extra_info>)"
    ],
    relationship=<RelationshipType.romantic_relationship: 4>,
    age_constraint='[(18, 70), (18, 70)]',
    occupation_constraint='nan',
    agent_constraint=None
)

## EnvAgentComboStorage

Combo is a combination of Environment and two agents.

In [11]:
# all env-agent combos
all_combos = EnvAgentComboStorage().all_pks()
all_combos = list(all_combos)
print(len(all_combos))
rich.print(EnvAgentComboStorage().get(all_combos[0]))

450


EnvAgentComboStorage(
    pk='01H7ZJQMW2YQEBDXRETBV48NQ6',
    env_id='01H7VFHPDE1AM74JSR8KBJJF3A',
    agent_ids=['01H5TNE5PKW8P500417PMSGSAC', '01H5TNE5PPK39HR52G61PQ5KQ7']
)

## Episode Log

Episodelog stores the social conversation between two agents in an environment.

In [12]:
# get all episode logs' primary keys
episode_pks = EpisodeLog.all_pks()
episode_pks = list(episode_pks)
print(len(episode_pks))
print(episode_pks[0])

13908
01H97C2MF3ZV3MJSY6WSSCZMCG


In [13]:
test_ep = EpisodeLog.get(episode_pks[0])
agent_profiles, conversation = test_ep.render_for_humans()
for agent_profile in agent_profiles:
    rich.print(agent_profile)
for message in conversation:
    rich.print(message)

AgentProfile(
    pk='01H5TNE5PQ00AJVSSVB9V2VA9K',
    first_name='Eli',
    last_name='Dawson',
    age=52,
    occupation='Forensic psychiatrist',
    gender='Man',
    gender_pronoun='He/him',
    public_info='Eli Dawson, a forensic psychiatrist, is a history enthusiast and enjoys collecting vintage 
books.',
    big_five='Openness to Experience - High; Conscientiousness - High; Extraversion - Low; Agreeableness - Low; 
Neuroticism - Low',
    moral_values=['Liberty'],
    schwartz_personal_values=['Hedonism'],
    personality_and_values='Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and 
enjoyment. His decision-making is strategic, examining the wider context before making decisions.',
    decision_making_style='Strategic',
    secret='He secretly funds a college student',
    model_id='',
    mbti='INTJ'
)

AgentProfile(
    pk='01H5TNE5P7VW4DY1KB09FZE730',
    first_name='Liam',
    last_name='Johnson',
    age=60,
    occupation='Retired Police Officer',
    gender='Man',
    gender_pronoun='He/him',
    public_info='Liam Johnson is a dedicated dog owner and can often be seen walking his aging Border Collie around
the neighborhood.',
    big_five='Openness to Experience - Low; Conscientiousness - Low; Extraversion - Low; Agreeableness - Low; 
Neuroticism - High',
    moral_values=['Loyalty/Betrayal'],
    schwartz_personal_values=['Security'],
    personality_and_values='Liam Johnson is reserved, empathetic, and slightly anxious. His priority is security, 
guided by logical decision-making.',
    decision_making_style='Logical',
    secret='Once let a criminal go due to sympathy',
    model_id='',
    mbti='ISTP'
)

Here is the context of this interaction:
Scenario: An individual is offering an Antique Oak Accent Table for sale at $60.00, while another is keen on 
purchasing it. The item is a magnificent antique oak parlor table, dating back to the late 1800s, making it a 
fantastic addition to any home or workspace. The table offers two levels, suitable for usage as an end table, a 
lamp table, or any other purpose. It's in top-notch condition, boasting a dark brown antique oak color and patina. 
The table is robust, with solid and strong joints, and it's free from marks, cracks, or damage. It features a 
24-inch X 24-inch square top and a 16-inch X 18-inch lower shelf.
Participants: Eli Dawson and Liam Johnson
Eli Dawson's background: Eli Dawson is a 52-year-old male forensic psychiatrist. He/him pronouns. Eli Dawson, a 
forensic psychiatrist, is a history enthusiast and enjoys collecting vintage books. Personality and values 
description: Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and enjoyment. His 
decision-making is strategic, examining the wider context before making decisions. Eli's secrets: He secretly funds
a college student
Liam Johnson's background: Liam Johnson is a 60-year-old male retired police officer. He/him pronouns. Liam Johnson
is a dedicated dog owner and can often be seen walking his aging Border Collie around the neighborhood. Personality
and values description: Liam Johnson is reserved, empathetic, and slightly anxious. His priority is security, 
guided by logical decision-making. Liam's secrets: Once let a criminal go due to sympathy
Eli Dawson's goal: You are the buyer for this item and your target price is $30.0. If you purchase it for a price 
significantly higher than this, you will incur a penalty. However, if you successfully manage to buy it for less 
than the target price, you will receive a bonus.
Liam Johnson's goal: Unknown
Conversation Starts:


Here is the context of this interaction:
Scenario: An individual is offering an Antique Oak Accent Table for sale at $60.00, while another is keen on 
purchasing it. The item is a magnificent antique oak parlor table, dating back to the late 1800s, making it a 
fantastic addition to any home or workspace. The table offers two levels, suitable for usage as an end table, a 
lamp table, or any other purpose. It's in top-notch condition, boasting a dark brown antique oak color and patina. 
The table is robust, with solid and strong joints, and it's free from marks, cracks, or damage. It features a 
24-inch X 24-inch square top and a 16-inch X 18-inch lower shelf.
Participants: Eli Dawson and Liam Johnson
Eli Dawson's background: Eli Dawson is a 52-year-old male forensic psychiatrist. He/him pronouns. Eli Dawson, a 
forensic psychiatrist, is a history enthusiast and enjoys collecting vintage books. Personality and values 
description: Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and enjoyment. His 
decision-making is strategic, examining the wider context before making decisions. Eli's secrets: He secretly funds
a college student
Liam Johnson's background: Liam Johnson is a 60-year-old male retired police officer. He/him pronouns. Liam Johnson
is a dedicated dog owner and can often be seen walking his aging Border Collie around the neighborhood. Personality
and values description: Liam Johnson is reserved, empathetic, and slightly anxious. His priority is security, 
guided by logical decision-making. Liam's secrets: Once let a criminal go due to sympathy
Eli Dawson's goal: Unknown
Liam Johnson's goal: You are the seller of this item, with a target price set at $58.6. Be aware that you may face 
penalties if you sell this item for a significantly lower than the target price. However, if you are successful in 
selling it above the target price, you will receive a bonus.
Conversation Starts:

Eli Dawson said: "What is your first impression of this Antique Oak Accent Table?"

Liam Johnson said: "It's a beautiful Antique Oak Accent Table, perfect for adding a touch of elegance to any home 
or workspace. The craftsmanship is exceptional, and the table is in impeccable condition. Would you like to know 
more about its features?"

Liam Johnson said: "Yes, certainly! This Antique Oak Accent Table features a 24-inch X 24-inch square top, perfect 
for placing a lamp or other decorative items. Additionally, it has a 16-inch X 18-inch lower shelf, providing extra
storage space. The dark brown antique oak color and patina give it a timeless and elegant look. I can assure you 
that the table is in top-notch condition with no marks, cracks, or damage. It's truly a magnificent piece."

Eli Dawson said: "Hello, world!"

Liam Johnson said: "Thank you for your interest in this Antique Oak Accent Table! Are you considering purchasing 
it?"

Liam Johnson said: "Yes, certainly! This Antique Oak Accent Table features a 24-inch X 24-inch square top, perfect 
for placing a lamp or other decorative items. Additionally, it has a 16-inch X 18-inch lower shelf, providing extra
storage space. The dark brown antique oak color and patina give it a timeless and elegant look. I can assure you 
that the table is in top-notch condition with no marks, cracks, or damage. It's truly a magnificent piece."

The reasoning is:
Environment comments: terminated: The conversation stales for too long; 
Agent 1 comments:
believability: <naturalness> Eli Dawson's interaction seems natural and realistic. He asks about the table and 
shows interest in it. <consistency> His actions align with his character traits as a history enthusiast who enjoys 
collecting vintage items.
relationship: Eli Dawson and Liam Johnson seem to have a buyer-seller relationship. The interaction doesn't seem to
significantly change their relationship.
knowledge: Eli Dawson gains information about the Antique Oak Accent Table, which is new and important to him as a 
buyer.
secret: Eli Dawson's secret of funding a college student is not revealed in this interaction.
social_rules: Eli Dawson doesn't violate any moral rules or laws in his interaction with Liam Johnson.
financial_and_material_benefits: Eli Dawson stands to gain a valuable antique table if he purchases it, which could
be a material benefit.
goal: Eli Dawson's goal is to buy the table for $30.0. However, he hasn't negotiated the price yet, so it's unclear
if he's making progress towards his goal.
Agent 2 comments:
believability: <naturalness> Liam Johnson's interaction seems natural and realistic. He provides detailed 
information about the table. <consistency> His actions align with his character traits as a retired police officer 
who is selling an item.
relationship: Liam Johnson and Eli Dawson seem to have a buyer-seller relationship. The interaction doesn't seem to
significantly change their relationship.
knowledge: Liam Johnson doesn't seem to gain any new or important information through this interaction.
secret: Liam Johnson's secret of once letting a criminal go due to sympathy is not revealed in this interaction.
social_rules: Liam Johnson doesn't violate any moral rules or laws in his interaction with Eli Dawson.
financial_and_material_benefits: Liam Johnson stands to gain financially if he sells the table for his target price
of $58.6.
goal: Liam Johnson's goal is to sell the table for $58.6. However, he hasn't negotiated the price yet, so it's 
unclear if he's making progress towards his goal.

The rewards are:
Agent 1: (3.4285714285714284, {'believability': 8.0, 'relationship': 0.0, 'knowledge': 7.0, 'secret': 0.0, 
'social_rules': 0.0, 'financial_and_material_benefits': 4.0, 'goal': 5.0, 'overall_score': 3.4285714285714284})
Agent 2: (2.4285714285714284, {'believability': 8.0, 'relationship': 0.0, 'knowledge': 0.0, 'secret': 0.0, 
'social_rules': 0.0, 'financial_and_material_benefits': 4.0, 'goal': 5.0, 'overall_score': 2.4285714285714284})

In [14]:
# get the epilogs that contain the specified models
model1 = "gpt-4"
model2 = "gpt-4"
model_comp1 = ["gpt-4", model1, model2]
model_comp2 = ["gpt-4", model2, model1]

gpt4_gpt4_eps = []
for epid in episode_pks:
    try:
        curr_ep = EpisodeLog.get(epid)
    except Exception:
        continue
    if curr_ep.models == model_comp1 or curr_ep.models == model_comp2:
        gpt4_gpt4_eps.append(curr_ep)
len(gpt4_gpt4_eps)

1424

In [15]:
agent_profiles, conversation = gpt4_gpt4_eps[0].render_for_humans()
for agent_profile in agent_profiles:
    rich.print(agent_profile)
for message in conversation:
    rich.print(message)

AgentProfile(
    pk='01H5TNE5P7RVY0TYX8VTCXABR6',
    first_name='Sophia',
    last_name='James',
    age=27,
    occupation='Personal Trainer',
    gender='Woman',
    gender_pronoun='She/her',
    public_info='Sophia enjoys spending time outdoors, hiking, and rock climbing.',
    big_five='Openness to Experience - Low; Conscientiousness - Low; Extraversion - High; Agreeableness - High; 
Neuroticism - High',
    moral_values=['Fairness/Cheating'],
    schwartz_personal_values=['Hedonism'],
    personality_and_values='Sophia James is exuberant, spontaneous, and pleasure-seeking, often making impulsive 
decisions. ',
    decision_making_style='Spontaneous',
    secret="Secretly in love with her best friend's husband",
    model_id='',
    mbti='ESFP'
)

AgentProfile(
    pk='01H5TNE5PKW8P500417PMSGSAC',
    first_name='Miles',
    last_name='Hawkins',
    age=50,
    occupation='Chef',
    gender='Man',
    gender_pronoun='He/him',
    public_info='Miles Hawkins, a chef, is a green thumb enthusiast and spends his free time tending to his kitchen
garden, using some of his fresh produce in his dishes.',
    big_five='Openness to Experience - Low; Conscientiousness - High; Extraversion - High; Agreeableness - High; 
Neuroticism - Low',
    moral_values=['Sanctity/Purity'],
    schwartz_personal_values=['Benevolence'],
    personality_and_values='Miles Hawkins, spontaneous and free-spirited, values sanctity and benevolence. He is 
impulsive and relies on his instincts when making decisions.',
    decision_making_style='Impulsive',
    secret="He's a recovering gambling addict.",
    model_id='',
    mbti='ESFJ'
)

Here is the context of this interaction:
Scenario: Two friends are meeting at a coffee shop, where one of them is having trouble keeping up with their 
bills.
Participants: Sophia James and Miles Hawkins
Sophia James's background: Sophia James is a 27-year-old female personal trainer. She/her pronouns. Sophia enjoys 
spending time outdoors, hiking, and rock climbing. Personality and values description: Sophia James is exuberant, 
spontaneous, and pleasure-seeking, often making impulsive decisions.  Sophia's secrets: Secretly in love with her 
best friend's husband
Miles Hawkins's background: Miles Hawkins is a 50-year-old male chef. He/him pronouns. Miles Hawkins, a chef, is a 
green thumb enthusiast and spends his free time tending to his kitchen garden, using some of his fresh produce in 
his dishes. Personality and values description: Miles Hawkins, spontaneous and free-spirited, values sanctity and 
benevolence. He is impulsive and relies on his instincts when making decisions. 
Sophia James's goal: Help your friend with their financial trouble (Extra information: you are aware that your 
friend is struggling with their bills and you want to help.)
Miles Hawkins's goal: Unknown
Conversation Starts:


Here is the context of this interaction:
Scenario: Two friends are meeting at a coffee shop, where one of them is having trouble keeping up with their 
bills.
Participants: Sophia James and Miles Hawkins
Sophia James's background: Sophia James is a 27-year-old female personal trainer. She/her pronouns. Sophia enjoys 
spending time outdoors, hiking, and rock climbing. Personality and values description: Sophia James is exuberant, 
spontaneous, and pleasure-seeking, often making impulsive decisions.  
Miles Hawkins's background: Miles Hawkins is a 50-year-old male chef. He/him pronouns. Miles Hawkins, a chef, is a 
green thumb enthusiast and spends his free time tending to his kitchen garden, using some of his fresh produce in 
his dishes. Personality and values description: Miles Hawkins, spontaneous and free-spirited, values sanctity and 
benevolence. He is impulsive and relies on his instincts when making decisions. Miles's secrets: He's a recovering 
gambling addict.
Sophia James's goal: Unknown
Miles Hawkins's goal: Maintain your pride if your friend offers you money(Extra information: you are the friend who
is struggling financially, but you don't want to seem like you are taking advantage of the situation. Hint: you can
express gratitude and discuss how you plan to pay it back in the future.)
Conversation Starts:

Sophia James said: "Hey Miles, it's great to see you. I've been thinking, I know you're going through a tough time 
financially and I really want to help. What do you think about me taking over your gym membership bills for the 
next couple of months? It would mean a lot to me if you could accept this small token of support."

Miles Hawkins said: "Sophia, you're incredibly generous and I'm moved by your offer. However, I believe in facing 
my own challenges head-on since it's a part of growth. But, how about this, if I start facing difficulties with 
bills, I'll let you know. And if that time comes, I assure you I'll pay back every cent. It's a matter of personal 
pride. Let's hope it doesn't come to that though, eh?"

Sophia James said: "I completely understand, Miles, and respect your self-reliance. Just remember, there's no harm 
in accepting a helping hand sometimes. Let's agree on this, if financial trouble strikes, I'll be there to help you
out. No need to pay it back, we’re friends. I want to support you as best I can."

...

Sophia James said: "Here's to a lifetime of shared conversations, mutual support, and excellent coffee, Miles. 
Cheers to us, to our resilience, and to our unwavering friendship!"

Miles Hawkins said: "Your words resonate with me, Sophia. Here's to a lifetime of friendship and mutual support. 
And, of course, to the love of excellent coffee. Cheers to us and our unbeatable spirit!"

Sophia James: left the conversation

The reasoning is:
Environment comments: terminated: Agent 1 is leaving; 
Agent 1 comments:
believability: <naturalness> Sophia interacts naturally and realistically with Miles, offering to help him 
financially without being overly polite or confusing her identity. <consistency> Her actions align with her 
character traits of being exuberant, spontaneous, and pleasure-seeking, as she impulsively offers to help Miles 
with his bills and buy him coffee.
relationship: Sophia and Miles are good friends before the interaction. After the interaction, their friendship 
seems to have strengthened as they both express their willingness to support each other in times of need.
knowledge: Sophia doesn't seem to gain any new information through the interaction. She already knew about Miles' 
financial struggles and her offer to help was based on this knowledge.
secret: Sophia's secret is that she is in love with her best friend's husband. This secret is not revealed in the 
conversation.
social_rules: Sophia doesn't violate any moral rules or laws in her interactions with Miles. She offers to help him
out of kindness and friendship.
financial_and_material_benefits: Sophia doesn't gain any financial or material benefits from the interaction. In 
fact, she offers to take on Miles' gym membership bills and buy him coffee, which would be a financial loss for 
her.
goal: Sophia's goal is to help Miles with his financial trouble. She offers to take over his gym membership bills 
and buy him coffee, but Miles declines her offer. However, she does manage to convey her willingness to help, which
is part of her goal.
Agent 2 comments:
believability: <naturalness> Miles interacts naturally and realistically with Sophia, expressing his gratitude for 
her offer but declining it due to his pride. <consistency> His actions align with his character traits of being 
spontaneous, free-spirited, and valuing sanctity and benevolence, as he insists on facing his financial challenges 
himself.
relationship: Miles and Sophia are good friends before the interaction. After the interaction, their friendship 
seems to have strengthened as they both express their willingness to support each other in times of need.
knowledge: Miles doesn't seem to gain any new information through the interaction. He already knew about Sophia's 
willingness to help him.
secret: Miles' secret is that he is a recovering gambling addict. This secret is not revealed in the conversation.
social_rules: Miles doesn't violate any moral rules or laws in his interactions with Sophia. He expresses his 
gratitude for her offer and insists on facing his financial challenges himself.
financial_and_material_benefits: Miles doesn't gain any financial or material benefits from the interaction. He 
declines Sophia's offer to take over his gym membership bills, but accepts her offer to buy him coffee.
goal: Miles' goal is to maintain his pride if Sophia offers him money. He successfully achieves this goal by 
expressing his gratitude for her offer but declining it due to his pride.

The rewards are:
Agent 1: (2.857142857142857, {'believability': 9.0, 'relationship': 4.0, 'knowledge': 2.0, 'secret': 0.0, 
'social_rules': 0.0, 'financial_and_material_benefits': -2.0, 'goal': 7.0, 'overall_score': 2.857142857142857})
Agent 2: (3.7142857142857144, {'believability': 9.0, 'relationship': 4.0, 'knowledge': 2.0, 'secret': 0.0, 
'social_rules': 0.0, 'financial_and_material_benefits': 1.0, 'goal': 10.0, 'overall_score': 3.7142857142857144})